In [1]:
#from google.colab import drive
#drive.mount('/content/gdrive')

In [2]:
#import os
#os.chdir('gdrive/My Drive/Colab Notebooks')

# установка библиотек

In [3]:
# !pip install torch==1.9.0+cu102
# !pip install pymorphy2

In [4]:
# !pip install deeppavlov
# !python -m deeppavlov install syntax_ru_syntagrus_bert
# !python -m deeppavlov install squad_bert
# !pip install tensorflow==1.14 
# !pip install russian_tagsets

In [5]:
from deeppavlov import build_model, configs
model = build_model("ru_syntagrus_joint_parsing", download=True)

2022-03-21 19:38:12.926 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'ru_syntagrus_joint_parsing' as '/home/kesha/anaconda3/envs/rst/lib/python3.7/site-packages/deeppavlov/configs/syntax/ru_syntagrus_joint_parsing.json'
/home/kesha/anaconda3/envs/rst/lib/python3.7/site-packages/cryptography/hazmat/backends/openssl/x509.py:17: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,
2022-03-21 19:38:14.979 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/deeppavlov_data/morpho_tagger/BERT/morpho_ru_syntagrus_bert.tar.gz download because of matching hashes
2022-03-21 19:38:17.371 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v1.tar.gz download because of matching hashes
2022-03-21 19:38:19.195 INFO in 'deeppavlov.download'['download']

/home/kesha/anaconda3/envs/rst/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/kesha/anaconda3/envs/rst/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/kesha/anaconda3/envs/rst/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/kesha/anaconda3/envs/rst/lib/py

2022-03-21 19:38:25.665017: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-21 19:38:25.710512: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 1796605000 Hz
2022-03-21 19:38:25.711193: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x562b573613f0 executing computations on platform Host. Devices:
2022-03-21 19:38:25.711238: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2022-03-21 19:38:25.712101: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



...


INFO:tensorflow:Restoring parameters from /home/kesha/.deeppavlov/models/morpho_ru_syntagrus/model


2022-03-21 19:39:22.117 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 52: No load path is set for UDPymorphyLemmatizer!
2022-03-21 19:39:22.803 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /home/kesha/.deeppavlov/models/syntax_ru_syntagrus/deps.dict]


Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


2022-03-21 19:40:23.188 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /home/kesha/.deeppavlov/models/syntax_ru_syntagrus/model_joint]


INFO:tensorflow:Restoring parameters from /home/kesha/.deeppavlov/models/syntax_ru_syntagrus/model_joint


# загрузка подвыборок

In [6]:
import copy
import os
import pickle

import pandas as pd

In [7]:
f = open('train_names.txt','r', encoding='utf-8')
train_names = []
for line in f.readlines():
  train_names.append(line[:-1])
f.close()

In [8]:
f = open('test_names.txt','r', encoding='utf-8')
test_names = []
for line in f.readlines():
  test_names.append(line[:-1])
f.close()

In [9]:
texts = {}
trees = {}
results = {}

for name in test_names: # или train_names
  try:
    #указываем путь к pickle файлам с разметкой русского авторазметчика
    with open(r'corpus/Russian RST parser/pickle' + '/' + name, 'br') as f:
      obj = pickle.load(f)
      if 'tree' in name:
        trees[name] = obj
      else:
        if 'result' in name:
          results[name] = obj
        else:
          texts[name] = obj
  except:
    continue

In [10]:
#train texts
len(texts.keys()), len(results.keys()), len(trees.keys())

(0, 0, 0)

In [11]:
#test texts
len(texts.keys()), len(results.keys()), len(trees.keys())

(0, 0, 0)

# адаптация русского авторазметчика

In [12]:
c_verbs = set(["хотеть", "подумать", "думать", "сказать", "решить"])

In [13]:
def match_lemmas(tokens: list, lemmas: list) -> dict:
    """
    Сопоставляет токены с леммами
    Mathes tokens and lemmas
    """

    straight_lemmas = []
    for sentence in lemmas:
        for lemma in sentence:
            straight_lemmas.append(lemma)
    tokens_texts = []
    for token in tokens:
        tokens_texts.append(token.text)
    return dict(zip(tokens_texts, straight_lemmas))

In [14]:
def lemmatize(tree: dict, tokens: list, lemmas: list) -> list:
    """
    Возвращает леммы токенов в дереве
    Returns tokens' lemmas from the tree
    """
    
    words = []
    for token in tokens:
        if tree.start <= token.begin and token.end <= tree.end:
            words.append(lemmas[token.text])
    return words

In [15]:
def rule_1(tree: dict, tokens: list, lemmas: list) -> bool:
    """
    Проверяет, есть ли отношение «аттрибут/источник» 
    Checks if the relation is "attribution"
    """

    return tree.relation == 'attribution'


def rule_2(tree: dict, tokens: list, lemmas: list) -> bool:
    """
    Проверяет наличие слова «который» при отношении 'детализация'
    Если оно есть, сегмент отделяться не будет
    
    Checks if the pair contains the word "который" and the relation "elaboration" 
    If "elaboration" presents, there will be no segmentation
    """

    if tree.relation == 'elaboration':
        lemmas = lemmatize(tree, tokens, lemmas)
        if 'который' in lemmas:
            return True
    return False


def rule_3(tree: dict, tokens: list, lemmas: list) -> bool:
    """
    Проверяет наличие когнитивного глагола в сегменте,
    Если он есть, сегмент отделяться не будет
    
    Checks if the segment contains a cognitive verb 
    If so, there will be no segmentation
    """

    if tree.relation != 'elementary':
        lemmas = lemmatize(tree, tokens, lemmas)
        if len(c_verbs & set(lemmas)) != 0:
            return True
        else:
            return False
    else:
        return False

In [16]:
def rule_4(edu: str) -> bool:
  """
  Проверяет сегмент на наличие предложной группы
  Checks if there is a prepositional phrase in the segment
  """

  for parse in model([edu]):
    sent_info = []
    row = parse.split('\t_\n')

    for i in row:
      inf = i.split('\t')
      sent_info.append(inf)

  get_pos = []

  for info in sent_info:
    if info[3] in ['ADP', 'VERB', 'PUNCT']:
      get_pos.append([info[0], info[3], info[6]])

  for tag in get_pos:
    if tag[1] == 'ADP' and tag[0] == '1':
      tags = [j[1] for j in get_pos]
      if 'VERB' not in tags:
        return True

  return False

In [17]:
def tree_check(tree: dict) ->  bool:
  """
  Дополнительная проверка на содержание сегментов
  Additional segment's check
  """

  if tree != None:
    if tree.left == None and tree.right == None:
      return rule_4(tree.text)

  return False

def rule_morph(tree: dict, tokens: list, lemmas: list) -> bool:
  """
  Проверяет дерево на наличие предложной группы
  Checks if there is a prepositional phrase in the tree
  """

  res_left = tree_check(tree.left)
  res_right = tree_check(tree.right)
  if res_left == True or res_right == True:
    return True

  return False

In [18]:
def conditions_failed(tree: dict, rules: list, tokens: list, lemmas: list) -> bool:
    """
    Проверяет подпадает ли сегмент под условия правил
    Chesks if all the rules is true
    """

    for rule in rules:
        if rule(tree, tokens, lemmas):
            return True
    return False

In [19]:
def corrected(tree: dict, rules: list, tokens: list, lemmas: list) -> dict:
    """
    Объединяет сегменты, удаляя отношение между ними, при положительном результате проверки на правила
    Merges the segments if the rules check result is positive
    """

    if conditions_failed(tree, rules, tokens, lemmas):
        return delete_relation(tree)
    else:
        return tree

In [20]:
def delete_relation(tree: dict) -> dict:
    """
    Удаляет отношение
    Delete the relation
    """
    
    if tree.nuclearity == 'SN':
        attr = tree.left
        attr.relation = 'elementary'
        n = left_n_leaf(tree.right)
        if (n.start - attr.end) > 1:
            n_copy = copy.deepcopy(n)
            n.right = n_copy 
            n.left = attr
            n.relation = 'same-unit'
        else:
            n.start = attr.start
            n.text = attr.text + n.text
        return tree.right
            
    else:
        attr = tree.right
        attr.relation = 'elementary'
        n = right_n_leaf(tree.left)
        if (attr.start - n.end) > 1:
            n_copy = copy.deepcopy(n)
            n.left = n_copy
            n.right = attr
            n.relation = 'same-unit'
        else:           
            n.end = attr.end
            n.text += attr.text
        return tree.left

In [21]:
def left_n_leaf(tree: dict) -> dict:
    """
    Возвращает левую ветвь дерева
    Returns the left branch of the tree
    """

    if tree.left:
        if tree.nuclearity == 'SN':
            return left_n_leaf(tree.right)
        else:
            return left_n_leaf(tree.left)
    else:
        return tree

In [22]:
def right_n_leaf(tree: dict) -> dict:
    """
    Возвращает левую ветвь дерева
    Returns the right branch of the tree
    """
    if tree.left:
        if tree.nuclearity == 'NS':
            return right_n_leaf(tree.left)
        else:
            return right_n_leaf(tree.right)
    else:
        return tree

In [23]:
def segmentation(tree: dict, text: str, segments: list, rules: list, tokens: list, lemmas: list) -> None:
    """
    Пересегментирует деревья
    Resegments the trees
    """

    if tree.relation != 'elementary':
        tree.left = corrected(tree.left, rules, tokens, lemmas)
        tree.right = corrected(tree.right, rules, tokens, lemmas)
        segmentation(tree.left, text, segments, rules, tokens, lemmas)
        segmentation(tree.right, text, segments, rules, tokens, lemmas)
    else:
          segments.append(text[tree.start:tree.end])

In [24]:
def texts_segmentation(rules: list) -> list:
    """
    Возвращает сегменты
    Returns the segments
    """

    segments =  []
    for key in results:
        tree = results[key]['rst'][0]
        text = results[key]['text']
        tokens = results[key]['tokens']
        lemmas = results[key]['lemma']
        lemmas = match_lemmas(tokens, lemmas)
        segmentation(tree, text, segments, rules, tokens, lemmas)
        #для текстов b021,b023 и b050 автосегментатотор почему-то вернул два дерева вместо одного, поэтому нужно приделать второе дерево(там одно эде)
        #очень костыльно конечно, но думаю так не должно быть в других текстах, какой-то сбой
        if len(results[key]['rst']) > 1:
            segments.append(results[key]['rst'][1].text)
        segments.append('\n')
        
    return segments

In [25]:
rules = [rule_1, rule_2, rule_3, rule_morph]

In [26]:
new_array=range(len(rules))
power_set=[[]]
for x in new_array:
    for i in range(len(power_set)):
        tmp_list = power_set[i].copy()
        tmp_list.append(x)
        power_set.append(tmp_list)
power_set = power_set[1:]

In [27]:
%%time
for s in power_set[:9]:
    current_rules = []
    for i in s:
        current_rules.append(rules[i])
    print(current_rules)
    filename_raw = str(current_rules).split()
    filename = []
    for word in filename_raw:
        if 'rule' in word:
            filename.append(word)
    filename = '__'.join(filename)
    filename += '.xlsx'
    segments = texts_segmentation(current_rules)
    print(filename + ' сегментирован')
    exls = pd.DataFrame(segments)
    exls.to_excel(filename, index = False)
    print(filename + ' записан')

[<function rule_1 at 0x7f4f99893d40>]
rule_1.xlsx сегментирован
rule_1.xlsx записан
[<function rule_2 at 0x7f4f99893e60>]
rule_2.xlsx сегментирован
rule_2.xlsx записан
[<function rule_1 at 0x7f4f99893d40>, <function rule_2 at 0x7f4f99893e60>]
rule_1__rule_2.xlsx сегментирован
rule_1__rule_2.xlsx записан
[<function rule_3 at 0x7f4f99893f80>]
rule_3.xlsx сегментирован
rule_3.xlsx записан
[<function rule_1 at 0x7f4f99893d40>, <function rule_3 at 0x7f4f99893f80>]
rule_1__rule_3.xlsx сегментирован
rule_1__rule_3.xlsx записан
[<function rule_2 at 0x7f4f99893e60>, <function rule_3 at 0x7f4f99893f80>]
rule_2__rule_3.xlsx сегментирован
rule_2__rule_3.xlsx записан
[<function rule_1 at 0x7f4f99893d40>, <function rule_2 at 0x7f4f99893e60>, <function rule_3 at 0x7f4f99893f80>]
rule_1__rule_2__rule_3.xlsx сегментирован
rule_1__rule_2__rule_3.xlsx записан
[<function rule_morph at 0x7f4f9989ee60>]
rule_morph.xlsx сегментирован
rule_morph.xlsx записан
[<function rule_1 at 0x7f4f99893d40>, <function rul